# Simulator

In [17]:
import sys
sys.path.insert(0, "..")
import pandas as pd
import matplotlib.pyplot as plt
from trading_simulator import TradingSimulator, DLSimulator
from dl_solutions.dlsolutions import CryptoDLSolutions
from data_processor.data_processing import DataProcessor

## Classic

In [4]:
cryptos = ['ADA', 'BTC', 'ETH', 'LTC', 'LNK']
sel = 0

In [5]:
for crypto in cryptos:
    processor = DataProcessor([crypto])
    sim = TradingSimulator(processor, crypto, strategy = [1, 2, 3, 4, 5, 6, 7], loss_allowed = 0.2, log = False)
    sim.simulate()

Loading... ADA
Total invested: 18100 €
Balance: 13172.688982503865 €
Orders won: 57
Orders lost: 122
Profit: 0.17528668411623566 

Loading... BTC


KeyboardInterrupt: 

In [3]:
processor = DataProcessor([crypto])
sim1 = TradingSimulator(processor, crypto, strategy = [1], loss_allowed = 0.2)
sim2 = TradingSimulator(processor, crypto, strategy = [2], loss_allowed = 0.2)
sim3 = TradingSimulator(processor, crypto, strategy = [3], loss_allowed = 0.2)
sim4 = TradingSimulator(processor, crypto, strategy = [4], loss_allowed = 0.2)
sim5 = TradingSimulator(processor, crypto, strategy = [5], loss_allowed = 0.2)
sim6 = TradingSimulator(processor, crypto, strategy = [6], loss_allowed = 0.2)
sim6 = TradingSimulator(processor, crypto, strategy = [7], loss_allowed = 0.2)
sim7 = TradingSimulator(processor, crypto, strategy = [1, 2, 3, 4, 5, 6, 7], loss_allowed = 0.2)

Loading... BTC
Reversing order
Dropping columns
Dropping Nan
Changing date format
Loading... BTC
Reversing order
Dropping columns
Dropping Nan
Changing date format
Loading... BTC
Reversing order
Dropping columns
Dropping Nan
Changing date format
Loading... BTC
Reversing order
Dropping columns
Dropping Nan
Changing date format
Loading... BTC
Reversing order
Dropping columns
Dropping Nan
Changing date format
Loading... BTC
Reversing order
Dropping columns
Dropping Nan
Changing date format
Loading... BTC
Reversing order
Dropping columns
Dropping Nan
Changing date format
Loading... BTC
Reversing order
Dropping columns
Dropping Nan
Changing date format


In [4]:
sim1.simulate()

Buy on 2019-04-24 00:00:00 ||| Strat 1  Close: 5415.0 | Stop-loss: 4332.0 | Take-profit: 99999999999999 |||
Sell order 0 on 2019-07-14 00:00:00  ||| Close: 10174.18 | Entry 5415.0 | Profit: 1.8788882733148662 | Balance: 10085.888827331486 |||
Buy on 2020-02-18 00:00:00 ||| Strat 1  Close: 10164.71 | Stop-loss: 8131.767999999999 | Take-profit: 99999999999999 |||
Sell order 0 on 2020-03-08 00:00:00  ||| Close: 8033.31 | Entry 10164.71 | Profit: 0.7903137423497573 | Balance: 10062.920201566461 |||
Buy on 2020-05-21 00:00:00 ||| Strat 1  Close: 9068.65 | Stop-loss: 7254.92 | Take-profit: 99999999999999 |||
Sell order 0 on 2021-01-21 00:00:00  ||| Close: 30850.13 | Entry 9068.65 | Profit: 3.4018437143345484 | Balance: 10301.104572999917 |||
Buy on 2021-09-15 00:00:00 ||| Strat 1  Close: 48121.41 | Stop-loss: 38497.128000000004 | Take-profit: 99999999999999 |||
Sell order 0 on 2021-11-26 00:00:00  ||| Close: 53726.53 | Entry 48121.41 | Profit: 1.1164787149836215 | Balance: 10310.75244449828 

In [5]:
sim2.simulate()

Buy on 2017-09-15 00:00:00 ||| Strat 2  Close: 3700.0 | Stop-loss: 2960.0 | Take-profit: 99999999999999 |||
Sell order 0 on 2017-11-12 00:00:00  ||| Close: 5811.03 | Entry 3700.0 | Profit: 1.5705486486486486 | Balance: 10055.054864864866 |||
Buy on 2017-11-13 00:00:00 ||| Strat 2  Close: 6465.99 | Stop-loss: 5172.7919999999995 | Take-profit: 99999999999999 |||
Sell order 0 on 2017-12-22 00:00:00  ||| Close: 13326.61 | Entry 6465.99 | Profit: 2.0610316440328553 | Balance: 10159.15802926815 |||
Buy on 2017-12-23 00:00:00 ||| Strat 2  Close: 13300.0 | Stop-loss: 10640.0 | Take-profit: 99999999999999 |||
Sell order 0 on 2017-12-30 00:00:00  ||| Close: 12440.01 | Entry 13300.0 | Profit: 0.935339097744361 | Balance: 10150.691939042586 |||
Buy on 2018-01-17 00:00:00 ||| Strat 2  Close: 10988.79 | Stop-loss: 8791.032000000001 | Take-profit: 99999999999999 |||
Sell order 0 on 2018-01-30 00:00:00  ||| Close: 10237.51 | Entry 10988.79 | Profit: 0.9316321451224383 | Balance: 10141.85515355483 |||


In [6]:
sim3.simulate()

Buy on 2017-09-06 00:00:00 ||| Strat 3  Close: 4619.77 | Stop-loss: 3695.8160000000003 | Take-profit: 99999999999999 |||
Sell order 0 on 2017-09-14 00:00:00  ||| Close: 3189.02 | Entry 4619.77 | Profit: 0.6902984347705621 | Balance: 9967.029843477056 |||
Buy on 2017-09-25 00:00:00 ||| Strat 3  Close: 3920.75 | Stop-loss: 3136.6 | Take-profit: 99999999999999 |||
Buy on 2017-10-31 00:00:00 ||| Strat 3  Close: 6463.0 | Stop-loss: 5170.4 | Take-profit: 99999999999999 |||
Sell order 0 on 2017-11-12 00:00:00  ||| Close: 5811.03 | Entry 3920.75 | Profit: 1.4821220429764712 | Balance: 9911.242047774704 |||
Sell order 1 on 2017-11-12 00:00:00  ||| Close: 5811.03 | Entry 6463.0 | Profit: 0.8991226984372582 | Balance: 10001.15431761843 |||
Buy on 2017-11-17 00:00:00 ||| Strat 3  Close: 7699.19 | Stop-loss: 6159.352 | Take-profit: 99999999999999 |||
Sell order 0 on 2017-12-22 00:00:00  ||| Close: 13326.61 | Entry 7699.19 | Profit: 1.7309106542376538 | Balance: 10072.245383042195 |||
Buy on 2018-01

In [7]:
sim4.simulate()

Buy on 2017-09-15 00:00:00 ||| Strat 4  Close: 3700.0 | Stop-loss: 2960.0 | Take-profit: 99999999999999 |||
Sell order 0 on 2017-11-12 00:00:00  ||| Close: 5811.03 | Entry 3700.0 | Profit: 1.5705486486486486 | Balance: 10055.054864864866 |||
Buy on 2017-11-13 00:00:00 ||| Strat 4  Close: 6465.99 | Stop-loss: 5172.7919999999995 | Take-profit: 99999999999999 |||
Sell order 0 on 2017-12-22 00:00:00  ||| Close: 13326.61 | Entry 6465.99 | Profit: 2.0610316440328553 | Balance: 10159.15802926815 |||
Buy on 2017-12-23 00:00:00 ||| Strat 4  Close: 13300.0 | Stop-loss: 10640.0 | Take-profit: 99999999999999 |||
Sell order 0 on 2017-12-30 00:00:00  ||| Close: 12440.01 | Entry 13300.0 | Profit: 0.935339097744361 | Balance: 10150.691939042586 |||
Buy on 2018-01-17 00:00:00 ||| Strat 4  Close: 10988.79 | Stop-loss: 8791.032000000001 | Take-profit: 99999999999999 |||
Sell order 0 on 2018-01-30 00:00:00  ||| Close: 10237.51 | Entry 10988.79 | Profit: 0.9316321451224383 | Balance: 10141.85515355483 |||


In [8]:
sim5.simulate()

Buy on 2017-08-25 00:00:00 ||| Strat 5  Close: 4280.68 | Stop-loss: 3424.5440000000003 | Take-profit: 99999999999999 |||
Buy on 2017-09-06 00:00:00 ||| Strat 5  Close: 4619.77 | Stop-loss: 3695.8160000000003 | Take-profit: 99999999999999 |||
Sell order 0 on 2017-09-14 00:00:00  ||| Close: 3189.02 | Entry 4280.68 | Profit: 0.7449797695693207 | Balance: 9870.497976956933 |||
Sell order 1 on 2017-09-14 00:00:00  ||| Close: 3189.02 | Entry 4619.77 | Profit: 0.6902984347705621 | Balance: 9939.527820433988 |||
Buy on 2017-09-25 00:00:00 ||| Strat 5  Close: 3920.75 | Stop-loss: 3136.6 | Take-profit: 99999999999999 |||
Buy on 2017-10-31 00:00:00 ||| Strat 5  Close: 6463.0 | Stop-loss: 5170.4 | Take-profit: 99999999999999 |||
Sell order 0 on 2017-11-12 00:00:00  ||| Close: 5811.03 | Entry 3920.75 | Profit: 1.4821220429764712 | Balance: 9883.740024731636 |||
Sell order 1 on 2017-11-12 00:00:00  ||| Close: 5811.03 | Entry 6463.0 | Profit: 0.8991226984372582 | Balance: 9973.652294575362 |||
Buy on

In [9]:
sim6.simulate()

Buy on 2021-03-25 00:00:00 ||| Strat 7  Close: 51293.78 | Stop-loss: 41035.024 | Take-profit: 99999999999999 |||
Sell order 0 on 2021-04-24 00:00:00  ||| Close: 50047.84 | Entry 51293.78 | Profit: 0.9757097254286972 | Balance: 9995.57097254287 |||
Buy on 2021-05-14 00:00:00 ||| Strat 7  Close: 49841.45 | Stop-loss: 39873.159999999996 | Take-profit: 99999999999999 |||
Sell order 0 on 2021-05-19 00:00:00  ||| Close: 36690.09 | Entry 49841.45 | Profit: 0.736136087533569 | Balance: 9967.184581296227 |||
Total invested: 200 €
Balance: 9967.184581296227 €
Orders won: 0
Orders lost: 2


In [10]:
sim7.simulate()

Buy on 2017-08-25 00:00:00 ||| Strat 5  Close: 4280.68 | Stop-loss: 3424.5440000000003 | Take-profit: 99999999999999 |||
Buy on 2017-09-06 00:00:00 ||| Strat 3  Close: 4619.77 | Stop-loss: 3695.8160000000003 | Take-profit: 99999999999999 |||
Buy on 2017-09-06 00:00:00 ||| Strat 5  Close: 4619.77 | Stop-loss: 3695.8160000000003 | Take-profit: 99999999999999 |||
Sell order 0 on 2017-09-14 00:00:00  ||| Close: 3189.02 | Entry 4280.68 | Profit: 0.7449797695693207 | Balance: 9768.497976956933 |||
Sell order 1 on 2017-09-14 00:00:00  ||| Close: 3189.02 | Entry 4619.77 | Profit: 0.6902984347705621 | Balance: 9837.527820433988 |||
Sell order 2 on 2017-09-14 00:00:00  ||| Close: 3189.02 | Entry 4619.77 | Profit: 0.6902984347705621 | Balance: 9906.557663911044 |||
Buy on 2017-09-15 00:00:00 ||| Strat 2  Close: 3700.0 | Stop-loss: 2960.0 | Take-profit: 99999999999999 |||
Buy on 2017-09-15 00:00:00 ||| Strat 4  Close: 3700.0 | Stop-loss: 2960.0 | Take-profit: 99999999999999 |||
Buy on 2017-09-15 0

## DL Solution

In [43]:
crypto = 'ETH'
periods_to_retraining = 1000
prev_periods = 2
pred_periods = 10
model_selector = 'lstm'
columns = ['weekday', 'RSI', 'MACD', 'close']

processor = DataProcessor([crypto])
sim = DLSimulator(processor, crypto, periods_to_retraining, prev_periods, pred_periods, model_selector, columns)
#sim.simulate()
df = sim.get_df()
real_df = df.iloc[14:1000]
to_pred_df = df.iloc[-1:]
real_and_pred = df.iloc[:1001]
real_df

Loading... ETH
Extracting columns columns for ETH
Proccessing and arranging columns for LSTM model


,close_1,MACD_1,RSI_1,weekday_1,close_0,MACD_0,RSI_0,weekday_0,close
139,754.99,-14.168903,51.874455,0.0,855.28,-3.037980,64.086633,1,1253.87
140,855.28,-3.037980,64.086633,1.0,934.03,11.999501,70.431522,2,1388.02
141,934.03,11.999501,70.431522,2.0,940.00,24.120497,70.851945,3,1347.00
142,940.00,24.120497,70.851945,3.0,959.30,34.881726,72.226781,4,1271.00
143,959.30,34.881726,72.226781,4.0,1004.11,46.489964,75.156684,5,1000.00
...,...,...,...,...,...,...,...,...,...
1120,337.39,-6.752410,39.923707,1.0,350.85,-8.430976,43.191875,2,385.25
1121,350.85,-8.430976,43.191875,2.0,367.93,-8.287505,47.122821,3,371.04
1122,367.93,-8.287505,47.122821,3.0,373.63,-7.625953,48.405914,4,340.36
1123,373.63,-7.625953,48.405914,4.0,387.78,-5.891963,51.549017,5,344.21


In [44]:
norm_strat = 0
strat = 1
layers = 4
batch_size = 64
epochs = 150

lstm = CryptoDLSolutions(real_df, norm_strat, strat, layers, batch_size, epochs)
lstm.build()
print(lstm.get_model().summary())
lstm.train()

Input shape: (936, 1, 8) (936,) (50, 1, 8) (50,)
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_17 (LSTM)               (None, 1, 50)             11800     
_________________________________________________________________
lstm_18 (LSTM)               (None, 1, 50)             20200     
_________________________________________________________________
dropout_13 (Dropout)         (None, 1, 50)             0         
_________________________________________________________________
lstm_19 (LSTM)               (None, 1, 50)             20200     
_________________________________________________________________
dropout_14 (Dropout)         (None, 1, 50)             0         
_________________________________________________________________
lstm_20 (LSTM)               (None, 50)                20200     
_________________________________________________________________
dropo

In [47]:
pred = lstm.predict()
print('pred:', pred)

testx (1, 1, 8)
[[[0.68948136 0.94094377 0.90512388 0.93273006 0.68225989 0.94046738
   0.90346263 0.93677948]]]
real [348.97]
Test RMSE: 2.647
pred: [351.61691716]


In [45]:
to_pred_df = df.iloc[1000:1001]

In [46]:
lstm.set_test(to_pred_df)

In [15]:
df[-2:]

,close_39,close_38,close_37,close_36,close_35,close_34,close_33,close_32,close_31,close_30,...,close_8,close_7,close_6,close_5,close_4,close_3,close_2,close_1,close_0,close
1547,3387.76,3417.73,3386.31,3515.27,3574.89,3587.47,3563.13,3574.82,3414.22,3542.12,...,4589.89,4603.35,4532.32,4474.24,4518.00,4612.59,4807.98,4732.33,4628.78,3709.27
1548,3417.73,3386.31,3515.27,3574.89,3587.47,3563.13,3574.82,3414.22,3542.12,3491.46,...,4603.35,4532.32,4474.24,4518.00,4612.59,4807.98,4732.33,4628.78,4719.00,3725.46


In [14]:
to_pred_df = pd.read_csv('/Users/gonzalozabala/Documents/Master IA/AIMaster/TFM/Code/crypto_bot/Datasets/ETH.csv', header=[1])
to_pred_df = pd.DataFrame(to_pred_df['close'].iloc[:10]).T
to_pred_df['y'] = 686
to_pred_df

,0,1,2,3,4,5,6,7,8,9,y
close,3725.46,3709.27,3630.19,3792.75,4037.23,4063.56,4094.83,4046.36,4111.41,3979.51,686


In [20]:
processor = DataProcessor(['ETH'])
processor.load_data()
processor.clean_data('ETH')
processor.feature_extraction('ETH')
#columns = ['close', 'Open Close Difference', 'Volume USDT' ,'Result']
columns = ['close']
df = processor.feature_selection('ETH', columns) 
df[-40:]

Loading... ETH
Extracting columns columns for ETH


,close
1559,4085.97
1560,4339.44
1561,4269.36
1562,4524.85
1563,4041.20
1564,4095.29
1565,4294.78
1566,4444.53
1567,4630.26
1568,4582.95


In [13]:
to_pred_df

,Result_39,Result_38,Result_37,Result_36,Result_35,Result_34,Result_33,Result_32,Result_31,Result_30,...,Result_8,Result_7,Result_6,Result_5,Result_4,Result_3,Result_2,Result_1,Result_0,Result
1163,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1,1.0


In [ ]:
'''
Estamos calculando mal la validacion pero esta funcionando
Investigar porque estamos calculando mal validacion

Cambiar test y!!!!!! en set_test
'''

In [26]:
df[1039:1059] # atentos al offset creado por null en close_0 close_1 etc

,close
1163,413.98
1164,409.11
1165,412.14
1166,405.86
1167,392.52
1168,403.45
1169,388.23
1170,387.13
1171,382.49
1172,386.46
